In [20]:
from src.utils.utils import get_CIFAR10_img_transformer, validate_monte_carlo_prediction
import torch 
from sklearn.metrics import f1_score
from src.utils.utils import AverageMeter, coverage_size, get_monte_carlo_prediction_sets
import torchvision
from src.models.cifar10_conv_model import Cifar10ConvModel



In [21]:
test_set = torchvision.datasets.CIFAR10(train=False, root='../data/processed', transform=get_CIFAR10_img_transformer())
test_loader =torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)

In [22]:
model = Cifar10ConvModel()
model.load_state_dict(torch.load("../models/trained_conv_cifar10_model.pt"))
model.eval()
model = torch.nn.DataParallel(model).cpu()

In [93]:
def validate_monte_carlo_prediction(val_loader, predictions_sets):
    """ Validate monte-carlor prediction sets

    Parameters
    ----------
    val_loader : object
        data loader object from the data loader module
    predictions_sets : ArrayLike
        monte-carlo prediction sets

    Returns
    ---------
    coverage : float
    set_size : float
    f1_score : float
    """
    size = AverageMeter('size')
    coverage = AverageMeter('coverage')
    f1_scores = AverageMeter('f1_scores')
    pred_set_loader = torch.utils.data.DataLoader(predictions_sets, batch_size=val_loader.batch_size,shuffle=False, collate_fn=lambda x: x)
    for S, (_, y) in zip(pred_set_loader, val_loader):
        top1s = []
        [top1s.append(p_set[0]) for p_set in S]
        S_tensor = [torch.tensor(sublist) for sublist in S]
        cov, sz = coverage_size(S_tensor, y)
        f1 = f1_score(y_true=y.detach().cpu(),
                    y_pred=top1s, average='macro', zero_division=0)
        coverage.update(cov)
        size.update(sz)
        f1_scores.update(f1)
    return coverage.avg, size.avg, f1_scores.avg

In [90]:
res = get_monte_carlo_prediction_sets(val_loader=test_loader, model=model, alpha=0.1, n_samples=len(test_set), n_classes=10, forward_passes=25, randomized=True)

In [79]:
pred_set_loader = torch.utils.data.DataLoader(res[1], batch_size=32,shuffle=False, collate_fn=lambda x: x)
# for x in pred_set_loader:
#     print(x)
#     break
# print(res[1][0])
for S, (_, y) in zip(pred_set_loader, test_loader):
    top1s = []
    [top1s.append(p_set[0]) for p_set in S]
    S_tensor = [torch.tensor(sublist) for sublist in S]
    cvg, sze = coverage_size(S_tensor, y)
    f1 = f1_score(y_true=y.detach().cpu(),
                    y_pred=top1s, average='macro', zero_division=0)
    print(f1)
    break

[3, 8, 8, 0, 6, 6, 9, 6, 3, 1, 0, 9, 5, 7, 9, 6, 5, 9, 8, 6, 9, 0, 4, 9, 4, 6, 7, 0, 9, 6, 6, 5]
[[3, 5], [8], [8, 0, 9], [0, 8], [6], [6], [9, 1, 7], [6, 2, 4, 3], [3, 5], [1, 9], [0, 5, 2, 3, 8], [9], [5, 4, 3, 6], [7, 5], [9], [6, 3], [5], [9, 7], [8, 9], [6], [9, 1, 7, 0], [0], [4, 2, 3, 5, 0, 6], [9], [4, 2], [6, 3, 2, 5, 4], [7, 5, 3, 2, 4], [0, 4, 7], [9], [6], [6, 2], [5, 3, 2]]
0.6865367965367966


In [94]:
out = validate_monte_carlo_prediction(val_loader=test_loader, predictions_sets=res[1])
out

(0.8812899361022364, 2.1071285942492013, 0.65328483387119)

In [65]:
l1 = [[3, 5], [8], [8, 0, 9], [0, 8], [6], [6], [9, 1, 7], [6, 2, 4, 3], [3, 5], [1, 9], [0, 5, 2, 3, 8], [9], [5, 4, 3, 6], [7, 5], [9], [6, 3], [5], [9, 7], [8, 9], [6], [9, 1, 7, 0], [0], [4, 2, 3, 5, 0, 6], [9], [4, 2], [6, 3, 2, 5, 4], [7, 5, 3, 2, 4], [0, 4, 7], [9], [6], [6, 2], [5, 3, 2]]
l2 = [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5]

# Flatten l2
def calculate_percentage(l1, l2):
    c = 0
    for sublist1, sublist2 in zip(l1, l2):
        if  sublist2 in sublist1 : 
            c+=1
    return c/len(l1)


In [66]:
calculate_percentage(l1, l2)

0.9375

In [67]:
test_loader.batch_size

32